In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# you can obtain the model from https://www.kaggle.com/models/google/universal-sentence-encoder/tensorFlow1/lite
# pick the Tensorflow1 lite version model

Mounted at /content/drive


In [ ]:
import os

# this is necessary for tensorflowjswizard
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# Import & Model load

In [ ]:
use_model_path = '/content/drive/MyDrive/ndev-task-tracker/universal-sentence-encoder-tensorflow1-lite-v2'

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import sentencepiece as spm
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
print(tf.__version__)

2.18.0


In [ ]:
embed = hub.load(use_model_path)
sp = spm.SentencePieceProcessor()
sp.load(f"{use_model_path}/assets/universal_encoder_8k_spm.model")

True

# Data Preparation

In [ ]:
import os

# NOTE: dataset size are not affecting the model size (explicitly)

valid_title = [
    "project sprint 1",
    "project sprint 5",
    "project sprint 1.1",
    "project sprint 5.1",
    "taskhive dev",
    "Prime.com Dev",
    "ndev task tracker",
    "spenicle v1",
    "zenventory",
    "flownest",
    "contact app",
    "property web app",
    "pronews",
    "project sprint 5 beta",
    "project sprint 5 milestone",
    "DataSync Sprint 24 - Planning",
    "NotificationEngine v1.3 - Multi-Cloud Strategy",
    "NotificationEngine v11.3 - Deployment cost optimization"
    "NotificationEngine v2 - Mobile Push Implementation",
    "AppEngine 3 - Base",
    "AppEngine 3 - Dev",
    "AppEngine 3 - Map Integration",
    "NotificationEngine v2 - Mobile Push Implementation",
    "glassdoor app - seo improvement",
    "glassdoor - Base",
    "glassdoor app - Dev",
    "glassdoor - Map Integration",
]

meetings = [
    "project backlog refinement",
    "planning sprint",
    "technical debt sprint planning"
    "sprint retrospective session",
    "designing system architecture",
    "project discussion",
    "soda",
    "1:1 with manager",
    "attending a meeting",
    "developer catchup",
    "daily standup",
    "daily meeting",
    "internal meeting",
    "running daily standup",
    "meeting with mr colleague",
    "meet with mrs jane",
    "daily scrum meeting",
    "brainstorming session",
    "weekly sync up",
    "In-person daily meeting",
    "team meeting",
    "client meeting",
    "client call",
    "client feedback discussion",
    "client introduction",
    "lunch with client",
    "one on one with client",
    "call with client",
    "quarterly review",
    "stand-up meeting",
    "sync meeting",
    "discussion with the team",
    "meeting with stakeholders",
    "project review meeting",
    "technical discussion",
    "design review meeting",
    "planning meeting for next quarter",
    "client demo session",
    "internal team sync",
    "meeting with product manager",
    "daily huddle",
    "weekly planning session",
    "weekly progress demo",
    "progress demonstration",
    "Team - Junior Developer Mentoring",
    "Team - Tech Leadership Sync",
    "Team - Technical Interview",
    "Team - Q3 Roadmap Presentation",
    "Team - Tech Talk Organization",
    "Team Mentoring Session",
    "Emerging Tech Evaluation Workshop",
    "Technical Mentorship Program",
    "discussing roadmap",
    "discussing project roadmap",
]

# task that the result, is not known immediately by the user
background_task = [
    "Test backup and recovery systems",
    "setting up ci/cd",
    "configuring docker",
    "writing api spec",
    "deploying app",
    "app deployment",
    "refactoring code",
    "writing documentation",
    "Create and document engineering standards and guidelines",
    "resolving merge conflicts",
    "committing changes",
    "optimizing database",
    "setting up continuous integration and deployment pipelines",
    "docker container configuration",
    "api contract documentation",
    "application deployment to production",
    "codebase refactoring",
    "technical documentation creation",
    "resolving git merge conflicts",
    "pushing code to remote repository",
    "committing code changes",
    "project backlog grooming",
    "configuring build automation",
    "writing user guides",
    "resolving version control conflicts",
    "deploying a new service",
    "improving database query performance",
    "Implement preventive measures",
    "reduce app bundle size",
    "review apps bundle size",
    "writing comprehensive unit tests for a module",
    "investigating and fixing a memory leak issue",
    "investigating and fixing memory issues in frontend components",
    "writing unit tests",
    "works on error tracking implementation",
    "developing a reusable component for a new page",
    "ensure all the component are reusable",
    "ensure the component are easily sharable",
    "works on data migration scripts",
    "updating technical specifications",
    "updating design specifications",
    "addressing static analysis warnings",
    "Ensure GDPR and SOC2 compliance in new features",
    "fixing linter issues",
    "updating documentation",
]

# task that usually not have a ticket, but we do that to improve the app anyway
general_tasks = [
    "writing code",
    "fixing bugs",
    "reviewing prs",
    "managing backlog",
    "managing tickets",
    "monitor app usage tracking",
    "monitor user behaviour",
    "monitor app traffic",
    "Research emerging technologies",
    "conducting technical research",
    "Review microservices architecture design",
    "reviewing code",
    "Review and update business continuity procedures",
    "pair programming",
    "resolving bugs",
    "preparing presentation",
    "presenting updates",
    "collaborating on design",
    "writing proposal",
    "self explore",
    "self research",
    "research and found issue with memory optimization in react components",
    "bug research",
    "debugging application errors",
    "conducting code reviews",
    "help teammates's work",
    "creating slides for presentation",
    "providing project updates",
    "collaborating on system design",
    "writing a technical proposal",
    "exploring new technologies for a project",
    "researching software bugs",
    "debugging memory leak",
    "benchmarking app",
    "conducting performance benchmarking for the application",
    "investigating memory issues in frontend components",
    "investigating a memory leak issue",
    "investigate calendar component issue",
    "Analyze team velocity and plan resource allocation",
]

general_activities = [
    "afternoon prayer",
    "watching a movie",
    "playing games",
    "eating out",
    "scrolling tiktok",
    "cooking dinner",
    "taking a nap",
    "chatting with friends",
    "binge-watching netflix",
    "reading a novel",
    "doing laundry",
    "shopping online",
    "going to the mall",
    "napping",
    "watching youtube",
    "cleaning room",
    "going for a walk",
    "checking social media",
    "ordering food",
    "scrolling instagram",
    "taking a break",
    "technical training",
    "english class",
    "compliance training session",
    "web development security training",
    "training assesment",
    "prayer time in the afternoon",
    "watching a film for leisure",
    "playing video games for relaxation",
    "dining at a restaurant",
    "browsing short videos on tiktok",
    "preparing a meal for dinner",
    "taking a short rest",
    "talking with colleagues informally",
    "watching multiple episodes on netflix",
    "reading a fiction book",
    "doing the weekly laundry",
    "purchasing items online",
    "visiting a shopping mall",
    "getting some sleep",
    "watching videos on youtube for entertainment",
    "tidying up the living space",
    "going for a short walk outside",
    "checking updates on social media platforms",
    "ordering food for delivery",
    "browsing photos and videos on instagram",
    "taking a short pause from work",
    "attending a technical skill building session",
    "participating in an English language course",
    "completing mandatory compliance training",
    "attending a web security workshop",
    "undergoing a training evaluation"
]

# can be useful when no valid_title detected
# this is a task title that not representing any specific project/client
# usually task that the effect can be known by the user immediately (opposite of background task)
project_tasks = [
    "working on a new ui design",
    "create new component for new page",
    "resolve feedback: update the formatData approach",
    "resolve feedback from mr. xxx",
    "working on a new feature",
    "working on the new drawer",
    "adjust the page gdpr message",
    "handle form submit behaviour",
    "works on progressive web app features",
    "works on user interface improvements",
    "works on mobile responsive design",
    "designing a new user interface for the application",
    "addressing feedback on the data formatting logic",
    "implementing changes based on feedback from a team member",
    "building a new feature based on requirements",
    "working on the implementation of a new navigation drawer",
    "modifying the gdpr consent message displayed on the page",
    "implementing the logic for form submission and validation",
    "works on analytics dashboard",
    "Organize tech talk series and knowledge sharing initiatives",
    "Review data retention policies",
    "Cost-benefit analysis for service migration",
    "Lead incident response and post-mortem analysis",
    "Define technology roadmap for next 18 months",
    "Evaluate microservices vs monolith trade-offs",
    "Design unified architecture",
    "database performance optimization",
    "Distributed Systems Optimization",
    "App Infrastructure Optimization",
    "Edge Computing Architecture",
    "Cloud Native Transformation",
    "Platform Scalability Engineering",
    "Advanced Database Architecture",
    "Engineering Culture Evolution",
    "Future Technology Roadmap",
    "Technical Advisory Role",
    "Technical Thought Leadership",
    "Technology Patent Strategy",
    "Technical Communication Strategy",
    "Technical Excellence Program",
    "AI Integration Strategy",
    "Developer Experience Enhancement",
    "Technical Debt Strategy Summit",
    "Technology Risk Assessment",
]

dataset_dict = {
    "valid_title": valid_title,
    "background_task": background_task,
    "meetings": meetings,
    "general_tasks": general_tasks,
    "general_activities": general_activities,
    "project_tasks": project_tasks,
}

In [ ]:
valid_title += [
    "project sprint 6",
    "project sprint 6.2",
    "project sprint 7 beta",
    "taskhive sprint 2",
    "Prime.com Release v2",
    "ndev sprint 3 - QA prep",
    "spenicle v2 roadmap",
    "zenventory sprint 5",
    "flownest analytics",
    "contact app refactor",
    "property app v3",
    "pronews redesign",
    "DataSync Sprint 25 - Review",
    "DataSync Sprint 26 - Planning",
    "NotificationEngine v12.0 - Performance Tuning",
    "NotificationEngine v3 - Webhooks Integration",
    "AppEngine 4 - Release",
    "AppEngine 5 - Sprint 1",
    "glassdoor app - Accessibility Review",
    "glassdoor - Sprint Planning",
    "glassdoor - Release v1.1",
    "DataHub Platform - v2",
    "OpsPortal v4 - Incident Dashboard",
    "InsightX v3 - Metrics Pipeline",
    "SupportTool v1 - Admin Panel",
    "MonitoringTool v2 - Alert Integration",
    "AdminApp - Role Management",
    "EcomX Platform - Checkout Refactor",
    "Analytics Dashboard - Sprint 3",
    "Payments Module - v1.2",
    "CRM System - Lead Scoring",
    "InternalTools v3 - API Migration",
    "infra-scripts repo - Cleanup",
    "QA Automation - Sprint 4",
    "deployment-ops - Infra Sprint",
    "microservice-utils - Package Update",
]

## Back-translation

english -> france -> english

warning: this section can take long time

this is not required to run, as the original dataset is quite sufficient

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load models for English → French and French → English
en_to_fr_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
en_to_fr_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

fr_to_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
fr_to_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(text, tokenizer, model):
    inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def back_translate(text):
    """
    Performs back-translation from English to French and then back to English.

    Returns:
        str: The back-translated English text.
    """
    fr_text = translate(text, en_to_fr_tokenizer, en_to_fr_model)
    back_to_en = translate(fr_text, fr_to_en_tokenizer, fr_to_en_model)
    return back_to_en

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

In [ ]:
import pickle
import os

pickle_filename = "back_translated_data.pkl"

if os.path.exists(pickle_filename):
    print(f"Loading back-translated data from {pickle_filename}")
    with open(pickle_filename, 'rb') as f:
        back_translated_data_dict = pickle.load(f)
    background_task_bt = back_translated_data_dict["background_task_bt"]
    meetings_bt = back_translated_data_dict["meetings_bt"]
    general_tasks_bt = back_translated_data_dict["general_tasks_bt"]
    general_activities_bt = back_translated_data_dict["general_activities_bt"]
    project_tasks_bt = back_translated_data_dict["project_tasks_bt"]
else:
    print("Back-translated data not found. Performing back-translation...")
    background_task_bt = []
    meetings_bt = []
    general_tasks_bt = []
    general_activities_bt = []
    project_tasks_bt = []

    # Back-translate sentences for each category (excluding valid_title)
    for sentence in background_task:
        background_task_bt.append(back_translate(sentence))

    for sentence in meetings:
        meetings_bt.append(back_translate(sentence))

    for sentence in general_tasks:
        general_tasks_bt.append(back_translate(sentence))

    for sentence in general_activities:
        general_activities_bt.append(back_translate(sentence))

    for sentence in project_tasks:
        project_tasks_bt.append(back_translate(sentence))

    # Create a dictionary to store the back-translated data
    back_translated_data_dict = {
        "background_task_bt": background_task_bt,
        "meetings_bt": meetings_bt,
        "general_tasks_bt": general_tasks_bt,
        "general_activities_bt": general_activities_bt,
        "project_tasks_bt": project_tasks_bt
    }

    # Save the dictionary to a pickle file
    with open(pickle_filename, 'wb') as f:
        pickle.dump(back_translated_data_dict, f)

    print(f"Back-translated data saved to {pickle_filename}")

print("Back-translated background_tasks:", background_task_bt[:5])
print("Back-translated meetings:", meetings_bt[:5])
print("Back-translated general_tasks:", general_tasks_bt[:5])
print("Back-translated general_activities:", general_activities_bt[:5])
print("Back-translated project_tasks:", project_tasks_bt[:5])

Back-translated data not found. Performing back-translation...
Back-translated data saved to back_translated_data.pkl
Back-translated background_tasks: ['Back-up and test recovery systems', 'implementation ci/cd', 'Configuring Docker', 'writing api spec', 'deployment of the application']
Back-translated meetings: ['Improvement of project backlog', 'planning sprint', 'Technical sprint of debt planningsprint retrospective session', 'design of the system architecture', 'Discussion on the draft']
Back-translated general_tasks: ['Writing code', 'Bug fix', 'review of practices', 'management of the backlog', 'banknote management']
Back-translated general_activities: ['afternoon prayer', 'Watch a movie', 'play games', 'Eat outside', 'tiktok scrolling']
Back-translated project_tasks: ['working on a new design ui', 'create a new component for a new page', 'resolve feedback: update DataApproche format', 'resolve feedback from mr. xxx', 'working on a new feature']


In [ ]:
print(len(background_task_bt))
print(len(meetings_bt))
print(len(general_tasks_bt))
print(len(general_activities_bt))
print(len(project_tasks_bt))

print(len(background_task))
print(len(meetings))
print(len(general_tasks))
print(len(general_activities))
print(len(project_tasks))


45
53
39
52
44
45
53
39
52
44


In [ ]:
import pickle

# Create a dictionary to store the back-translated data
back_translated_data_dict = {
    "background_task_bt": background_task_bt,
    "meetings_bt": meetings_bt,
    "general_tasks_bt": general_tasks_bt,
    "general_activities_bt": general_activities_bt,
    "project_tasks_bt": project_tasks_bt
}

# Define the filename for the pickle file
pickle_filename = "back_translated_data.pkl"

# Save the dictionary to a pickle file
with open(pickle_filename, 'wb') as f:
    pickle.dump(back_translated_data_dict, f)

print(f"Back-translated data saved to {pickle_filename}")

Back-translated data saved to back_translated_data.pkl


In [ ]:
# Combine original and back-translated data
background_task.extend(back_translated_data_dict["background_task_bt"])
meetings.extend(back_translated_data_dict["meetings_bt"])
general_tasks.extend(back_translated_data_dict["general_tasks_bt"])
general_activities.extend(back_translated_data_dict["general_activities_bt"])
project_tasks.extend(back_translated_data_dict["project_tasks_bt"])

## Preprocessing

In [ ]:
embed_fn = embed.signatures["default"]

In [ ]:
# USE Lite is designed to be smaller and mobile/web-friendly, so it does not contain its own tokenizer.
# because the embed_fn signature of the Universal Sentence Encoder Lite model you loaded expects the input in this sparse format. If you tried to pass a dense tensor or just a list of token IDs directly, the model would likely throw an error.

def to_sparse(sentences):
    # Encode sentences to list of token ids
    ids = [sp.encode(s) for s in sentences]

    # Create values and indices for SparseTensor
    values = [token for sent in ids for token in sent]
    indices = [[i, j] for i, sent in enumerate(ids) for j in range(len(sent))]
    dense_shape = [len(ids), max(len(sent) for sent in ids)]

    # Convert to required tensors
    return {
        "values": tf.constant(values, dtype=tf.int64),
        "indices": tf.constant(indices, dtype=tf.int64),
        "dense_shape": tf.constant(dense_shape, dtype=tf.int64),
    }


In [ ]:
# for evaluation
class_names = [
    "valid_title",
    "background_task",
    "meetings",
    "general_tasks",
    "general_activities",
    "project_tasks",
]

categories = [
    valid_title,
    background_task,
    meetings,
    general_tasks,
    general_activities,
    project_tasks
]

raw_data = [(item, label) for label, category in enumerate(categories) for item in category]

print('You have ', len(raw_data), 'data points')

You have  528 data points


In [ ]:
sentences = []
labels = []

for sentence, label in raw_data:
    sentences.append(sentence.lower())
    labels.append(label)

sentences = np.array(sentences)
labels = np.array(labels)

# Display the first few elements to verify
print(f"First 5 sentences: {sentences[:5]}")
print(f"First 5 labels: {labels[:5]}")

First 5 sentences: ['project sprint 1' 'project sprint 5' 'project sprint 1.1'
 'project sprint 5.1' 'taskhive dev']
First 5 labels: [0 0 0 0 0]


In [ ]:
sparse_input = to_sparse(sentences)
embeddings = embed_fn(**sparse_input)['default']

## Data splitting

In [ ]:
# Step 1: First split off the test set (15%)
x_temp, x_test, y_temp, y_test = train_test_split(
    embeddings.numpy(), labels, test_size=0.15, stratify=labels, random_state=7
)

# Step 2: Then split train and validation (from remaining 85%)
# Validation should be 15/85 = ~17.6% of x_temp
x_train, x_val, y_train, y_val = train_test_split(
    x_temp, y_temp, test_size=0.176, stratify=y_temp, random_state=7
)


In [ ]:
import collections

# Check if y_test have proper distribution
y_test_counts = collections.Counter(y_test)

# Print the counts for each class
print("Distribution of classes in y_val:")
for label, count in y_test_counts.items():
    print(f"Class {class_names[label]}: {count}")

Distribution of classes in y_val:
Class background_task: 14
Class meetings: 16
Class general_activities: 16
Class general_tasks: 12
Class valid_title: 9
Class project_tasks: 13


## prod setup

In [ ]:
# combine all dataset for final result

# x_train = np.concatenate((x_train, x_val), axis=0)
# y_train = np.concatenate((y_train, y_val), axis=0)
# x_train = np.concatenate((x_train, x_test), axis=0)
# y_train = np.concatenate((y_train, y_test), axis=0)

# len(x_train)

In [ ]:
x_train = tf.math.l2_normalize(x_train, axis=1)

# Training

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(512,)),
    tf.keras.layers.Dense(12, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(6, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# considering a small dataset, and class imbalance
# target ~75% acc, and .8 loss (for validation data)
# if back-translation applied, the dataset doubled
# then we can aim for 80%+ acc
# and valid_title class prediction accuracy should be prioritized
# try to get 15-20% validation data if possible
# history = model.fit(x_train, y_train, epochs=200, validation_split=0.2, callbacks=[early_stop])
# use manual split for better control
model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=300,
    callbacks=[early_stop]
)

# this reset if dataaset updated
# note (hidden layer (n) x dropout x epoch)
# 12-.3-300, 82%

Epoch 1/300
12/12 [==============================] - 1s 22ms/step - loss: 1.7773 - accuracy: 0.2575 - val_loss: 1.7460 - val_accuracy: 0.4051
Epoch 2/300
12/12 [==============================] - 0s 6ms/step - loss: 1.7366 - accuracy: 0.3279 - val_loss: 1.7020 - val_accuracy: 0.4051
Epoch 3/300
12/12 [==============================] - 0s 4ms/step - loss: 1.6963 - accuracy: 0.3604 - val_loss: 1.6602 - val_accuracy: 0.4304
Epoch 4/300
12/12 [==============================] - 0s 6ms/step - loss: 1.6442 - accuracy: 0.3848 - val_loss: 1.6152 - val_accuracy: 0.4304
Epoch 5/300
12/12 [==============================] - 0s 6ms/step - loss: 1.6072 - accuracy: 0.4119 - val_loss: 1.5734 - val_accuracy: 0.4177
Epoch 6/300
12/12 [==============================] - 0s 6ms/step - loss: 1.5710 - accuracy: 0.3984 - val_loss: 1.5303 - val_accuracy: 0.4684
Epoch 7/300
12/12 [==============================] - 0s 6ms/step - loss: 1.5297 - accuracy: 0.3984 - val_loss: 1.4879 - val_accuracy: 0.4557
Epoch 8/300


## Evaluation

In [ ]:
import matplotlib.pyplot as plt

# Extract training and validation metrics
history_dict = history.history
acc = history_dict.get('accuracy', [])
val_acc = history_dict.get('val_accuracy', [])
loss = history_dict.get('loss', [])
val_loss = history_dict.get('val_loss', [])

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'b-', label='Train Accuracy')
if val_acc:
    plt.plot(epochs, val_acc, 'r-', label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'b-', label='Train Loss')
if val_loss:
    plt.plot(epochs, val_loss, 'r-', label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'history' is not defined

## Acuracy

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Val Accuracy: {acc:.1%}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Predict class indices
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

# Visualize
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
# plt.legend()
plt.show()


In [ ]:
# Detailed per-class performance
print(classification_report(y_test, y_pred_classes, target_names=class_names))


## see wrong result

In [ ]:
new_sentences = ["my app v1", "works on container orchestration setup", "works on analytics dashboard", "eating a little pizza for a minute and wrote a code", "catchup with mr x", "Quarterly system architecture review meeting"]
new_sentences = project_tasks
new_sentences = ['my project v1']
new_input = to_sparse(new_sentences)
new_embeddings = embed_fn(**new_input)['default']
unseen_data_predictions = model.predict(new_embeddings)
predicted_labels_unseen = unseen_data_predictions.argmax(axis=1)

# Create a dictionary to store results
results_dict = {}
for i, label in enumerate(predicted_labels_unseen):
    class_name = class_names[label]
    if class_name not in results_dict:
        results_dict[class_name] = []
    results_dict[class_name].append(new_sentences[i])

print("Predicted classes for new sentences:")
display(results_dict)

# Save Model

In [ ]:
model.save('saved_model.h5')